In [1]:
!pip install datasets faiss-cpu==1.7.4 chromadb==0.4.22 sentence-transformers==2.3.1

In [2]:
!pip install -U datasets

In [3]:
from datasets import load_dataset

dataset=load_dataset("sadeem-ai/arabic-qna")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 1030
    })
})

In [5]:
dataset=dataset.filter(lambda example: example['has_answer'] == True)

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 4037
    })
    test: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 836
    })
})

In [7]:
text=dataset['train']['text']
text[0]

'اعتمادًا على المكان الذي ينتمي إليه الفرد، قد يكون تصورهم للوقت هو أن «الساعة تحكم اليوم» أو «سنصل إلى هناك عندما نصل إلى هناك». تحسين احتمالات النجاح في المجتمع العالمي يتطلب فهم الاختلافات الثقافية والتقاليد وأنماط الاتصال.'

In [8]:
metadata=[
    {
        "source":record['source'],
        "title":record['title']
    }

          for record in dataset['train']

]
metadata[0]

{'source': 'https://ar.wikipedia.org/wiki?curid=8976220',
 'title': 'تنظيم الوقت'}

In [9]:
ids=[
    str(i)
    for i in range(len(text))
]



In [10]:
from sentence_transformers import SentenceTransformer
model_id="sentence-transformers/distiluse-base-multilingual-cased-v2"
device="cuda:0"
dim=512
model=SentenceTransformer(model_id,device=device)

In [11]:
encode_doc=model.encode(text,show_progress_bar=True)

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

In [12]:
encode_doc.shape

(4037, 512)

In [13]:
len(text)

4037

In [14]:
text[0],encode_doc[0]

('اعتمادًا على المكان الذي ينتمي إليه الفرد، قد يكون تصورهم للوقت هو أن «الساعة تحكم اليوم» أو «سنصل إلى هناك عندما نصل إلى هناك». تحسين احتمالات النجاح في المجتمع العالمي يتطلب فهم الاختلافات الثقافية والتقاليد وأنماط الاتصال.',
 array([-2.22630110e-02,  1.18622780e-02, -5.17282784e-02, -3.84508967e-02,
         1.07650623e-01, -4.56619374e-02,  5.49685694e-02,  9.17534716e-03,
         3.52448076e-02,  6.96124462e-03,  3.18153016e-02,  2.13808799e-03,
        -9.56622139e-03, -6.63584918e-02, -4.39523384e-02,  8.27048048e-02,
         1.58673935e-02, -7.34318560e-03, -2.98200361e-02, -3.95070873e-02,
        -1.50665967e-02,  2.46608946e-02,  7.54381856e-03, -2.70484686e-02,
        -4.28158455e-02,  1.73906249e-03, -3.48085142e-03,  8.00499506e-03,
         1.49304494e-02, -1.12245372e-03, -8.69526118e-02,  2.12956294e-02,
         1.40109390e-03, -3.78256552e-02, -1.58398207e-02,  3.27752344e-02,
         4.64597298e-03,  5.33918887e-02,  1.11280270e-02,  1.02619836e-02,
        -2

In [15]:
!pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [16]:
import chromadb

chroma_client = chromadb.PersistentClient(path=r"D:\chroma-ar-docs")

In [17]:
collection=chroma_client.create_collection(
    name='Documnts',
    metadata={"hnsw:space":"cosine"}

)

In [20]:
collection.add(
    documents=text,
    embeddings=encode_doc,
    metadatas=metadata,
    ids=ids

)

In [22]:
question='ماذا كانت ترغب إسبانيا في استعادته في عام 1814؟'
question_embd=model.encode(question)
results=collection.query(
    query_embeddings=question_embd.tolist(),
    n_results=3

)
results

{'ids': [['139', '2752', '3178']],
 'distances': [[0.5236292481422424, 0.5821330547332764, 0.5821330547332764]],
 'metadatas': [[{'source': 'https://ar.wikipedia.org/wiki?curid=8393962',
    'title': 'إعلان الاستقلال الأرجنتيني'},
   {'source': 'https://ar.wikipedia.org/wiki?curid=3895', 'title': 'إسبانيا'},
   {'source': 'https://ar.wikipedia.org/wiki?curid=3895',
    'title': 'إسبانيا'}]],
 'embeddings': None,
 'documents': [['عندما استأنفت الملكية الإسبانية وظائفها في عام 1814، كانت إسبانيا مصممة على استعادة السيطرة على مستعمراتها في الأمريكتين. وعلاوة على ذلك، انتصر الملكيون من بيرو في معارك سيب سيب و واكي وفيلكابوجيو وأيوهوما في بيرو العليا، وهددوا المقاطعات المتحدة بشكل خطير من الشمال.',
   'غابت اليهودية من الناحية العملية عن إسبانيا منذ عام 1492 وحتى القرن التاسع عشر عندما تم السماح مرة أخرى لليهود بدخول البلاد. حالياً هناك حوالي 62,000 يهودي في إسبانيا أو 0,14% من مجموع السكان. كان معظم الوافدين في القرن الماضي في حين أن بعضهم من نسل اليهود الذين سكنوا إسبانيا سابقاً. يعتقد أن

In [24]:
import faiss
import numpy as np
from copy import deepcopy

In [25]:
norm_encode_docs=deepcopy(encode_doc)
faiss.normalize_L2(norm_encode_docs)

In [26]:
faiss_index=faiss.IndexIDMap(faiss.IndexFlatIP(dim))
faiss_index.add_with_ids(norm_encode_docs,ids)

In [27]:
question1='ماذا كانت ترغب إسبانيا في استعادته في عام 1814؟'
question_embd1=model.encode([question1])
faiss.normalize_L2(question_embd1)
result=faiss_index.search(question_embd1,3)
result

(array([[0.4763705, 0.4178667, 0.4178667]], dtype=float32),
 array([[ 139, 3178, 2752]]))

In [30]:
text[139]

'عندما استأنفت الملكية الإسبانية وظائفها في عام 1814، كانت إسبانيا مصممة على استعادة السيطرة على مستعمراتها في الأمريكتين. وعلاوة على ذلك، انتصر الملكيون من بيرو في معارك سيب سيب و واكي وفيلكابوجيو وأيوهوما في بيرو العليا، وهددوا المقاطعات المتحدة بشكل خطير من الشمال.'

In [35]:
import pickle

with open("./Faiss_arabic_document/index.pickle","wb") as handle:
  pickle.dump(faiss_index,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open("./Faiss_arabic_document/data.pickle","wb") as handle:
  pickle.dump({

          "text":text,
          "ids":ids,
          "metadata":metadata
      },handle,protocol=pickle.HIGHEST_PROTOCOL)


In [36]:
with open("./Faiss_arabic_document/index.pickle","rb") as handle:
  loaded_faiss_data=pickle.load(handle)

with open("./Faiss_arabic_document/data.pickle","rb") as handle:
  loaded_data=pickle.load(handle)